In [1]:
from keras.models  import load_model
import keras.backend as K

import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

import pandas as pd
import numpy as np

aspects = ['bit', 'domain']

Using TensorFlow backend.


[]


In [2]:
vec = cPickle.load(open('../../yelpdata/total_vec_120K_embed.p', 'rb'))

In [9]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../yelpdata/total_data_120K.csv')
aspect_columns = ['bit', 'domain']
                 
ds['bit'] = ds['stars'].apply(lambda x : x > 3.0)
train_idxs, val_idxs = train_test_split(ds.index, stratify=ds[['bit', 'domain']], train_size=0.9, random_state=1337)

idxs = val_idxs
H = {}
labs = {}
for aspect in aspect_columns :
    H[aspect] = np.zeros((len(idxs), len(idxs)))
    aspect_vals = ds[aspect].unique()
    labs[aspect] = np.array([0] * len(val_idxs))
    for i, val in enumerate(aspect_vals) :
        a = set(ds[ds[aspect] == val].index) & set(idxs)
        a = map(lambda s : list(idxs).index(s), a)
        labs[aspect][a] = i
        for j in a :
            H[aspect][j, a] = 1

    H[aspect][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [4]:
model = load_model('../store/weights/food/WedDec614:45:092017final1/loss.h5', 
                   custom_objects={'contrastive_loss' : lambda a,b:b})

In [5]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).inputs + [K.learning_phase()], 
                          model.get_layer('pred_'+aspect).get_layer('pool_'+aspect).outputs)

In [6]:
embeds = {}
X_s = vec.X[val_idxs]
vecs = {k:[] for k in aspects}
i, bs = 0, 100
while i*bs < len(val_idxs):
    for aspect in aspects :
        result = model_c1[aspect]([X_s[i*bs:(i+1)*bs], 0])[0]
        vecs[aspect].append(result)
    i += 1

for aspect in aspects :
    embeds[aspect] = np.concatenate(vecs[aspect], axis=0)


In [22]:
#Gated CNN
from sklearn.preprocessing import normalize
from sklearn.metrics import roc_auc_score
for aspect in aspects :
    result = embeds[aspect]
    result = normalize(result, 'l2')
    scores = np.dot(result, result.T)
    scores[np.arange(len(val_idxs)), np.arange(len(val_idxs))] = -1000
    aucs = [0] * len(val_idxs)
    aspect_j = aspect
#     for aspect_j in aspects :
    for i in range(len(val_idxs)) :
        v = labs[other_aspect[aspect]][i]
        o = np.where(labs[other_aspect[aspect]] != v)[0]
        aucs[i] = roc_auc_score(H[aspect_j][i, o], scores[i, o])
    print aspect, aspect_j, np.mean(aucs)

bit bit 0.917211351309
domain domain 0.988477641422


In [15]:
other_aspect = {'bit' : 'domain', 'domain' : 'bit'}

In [13]:
np.where(labs['bit'] != 0)[0]

array([    0,     3,     4, ..., 12702, 12703, 12707])